In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb

import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

plt.rcParams['font.sans-serif'] = ['Times New Roman']  # 用来正常显示中文标签
plt.rcParams["axes.unicode_minus"] = False  # 用来正常显示负号

import sklearn.decomposition
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer as LB
from imblearn.over_sampling import SMOTE, ADASYN

In [2]:
target_list=['Action','Adventure','Animated','Comedy','Historical','Fantasy','Horror','Science fiction','Documentary']
data=pd.read_csv(r'./data/data_clean_v2.csv')
data

,Extraversion,Observant,Thinking,Judging,Age,way of watching movies,Action,Adventure,Animated,Comedy,...,with_With friends,with_alone,with_with boyfriend/girlfriend,with_with family,Actor/actress,director,advertisements,genres,previews,comment
0,71,64,58,93,23.0,0,5,5,3,4,...,1,0,0,0,1.0,1.0,0.0,1.0,0.0,0.0
1,51,78,76,58,21.0,1,4,2,2,3,...,1,0,0,0,1.0,1.0,0.0,0.0,1.0,0.0
2,57,63,33,39,21.0,0,2,4,5,4,...,0,0,1,0,1.0,0.0,0.0,0.0,1.0,1.0
3,83,16,65,51,23.0,0,2,2,5,5,...,0,0,1,0,1.0,0.0,0.0,1.0,0.0,1.0
4,25,59,68,32,22.0,0,4,4,2,2,...,0,0,1,0,0.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,60,46,70,68,20.0,0,4,3,2,4,...,0,0,0,1,1.0,0.0,0.0,1.0,1.0,0.0
529,90,35,47,32,26.0,1,2,3,4,2,...,0,1,0,0,0.0,1.0,0.0,1.0,0.0,1.0
530,75,58,60,80,22.0,0,3,2,3,4,...,1,0,0,0,1.0,0.0,0.0,0.0,1.0,1.0
531,82,90,61,70,21.0,0,4,4,4,4,...,0,0,1,0,0.0,0.0,1.0,0.0,1.0,1.0


In [3]:
data_x=data.drop(columns=['Action','Adventure','Animated','Comedy','Historical','Fantasy','Horror','Science fiction','Documentary'],axis=1)
data_x_without=data_x.drop(['Extraversion','Observant','Thinking','Judging'],axis=1)
data_y=data[['Action','Adventure','Animated','Comedy','Historical','Fantasy','Horror','Science fiction','Documentary']]

In [4]:
MM=MinMaxScaler()
data_x[['Extraversion','Observant','Thinking','Judging','Age']]=MM.fit_transform(data_x[['Extraversion','Observant','Thinking','Judging','Age']])
data_x_without['Age']=(data_x_without['Age']-min(data_x_without['Age']))/(max(data_x_without['Age'])-min(data_x_without['Age']))

In [5]:
def MAPE(true, pred):
    nonzero = np.flatnonzero(true)
    true = true.ravel()[nonzero]
    pred = pred.ravel()[nonzero]
    diff = np.abs(np.array(true) - np.array(pred))
    mape = np.nanmean(diff / true)
    return mape

In [9]:
error_sum_rfr=0
error_rfr=[]
error_sum_without_rfr=0
error_without_rfr=[]
for i in target_list:
    x_train, x_test ,y_train,y_test=train_test_split(data_x,data_y[i],test_size=0.2, random_state=30,shuffle=True)
    rfr = RandomForestRegressor()
    x_train, y_train = ADASYN(sampling_strategy='minority').fit_resample(x_train,y_train)
    rfr.fit(x_train, y_train)
    rfr_y_predict = rfr.predict(x_test)
    error_sum_rfr+=MAPE(y_test,rfr_y_predict)
    error_rfr.append(MAPE(y_test,rfr_y_predict))
    x_train_wt, x_test_wt ,y_train_wt,y_test_wt=train_test_split(data_x_without,data_y[i],test_size=0.2, random_state=30,shuffle=True)
    rfr_without = RandomForestRegressor()
    x_train_wt, y_train_wt = ADASYN(sampling_strategy='minority').fit_resample(x_train_wt,y_train_wt)
    rfr_without.fit(x_train_wt, y_train_wt)
    rfr_y_without_predict = rfr_without.predict(x_test_wt)
    error_sum_without_rfr+=MAPE(y_test_wt,rfr_y_without_predict)
    error_without_rfr.append(MAPE(y_test_wt,rfr_y_without_predict))

In [10]:
print(error_sum_rfr/9)
print(error_sum_without_rfr/9)

0.42172343371408794
0.42312914503288335


In [31]:
error_sum_ad=0
error_ad=[]
error_sum_without_ad=0
error_without_ad=[]
for i in target_list:
    x_train, x_test ,y_train,y_test=train_test_split(data_x,data_y[i],test_size=0.2, random_state=30,shuffle=True)
    ad = AdaBoostRegressor()
    x_train, y_train = ADASYN(sampling_strategy='minority').fit_resample(x_train,y_train)
    ad.fit(x_train, y_train)
    ad_y_predict = ad.predict(x_test)
    error_sum_ad+=MAPE(y_test,ad_y_predict)
    error_ad.append(MAPE(y_test,ad_y_predict))
    x_train_wt, x_test_wt ,y_train_wt,y_test_wt=train_test_split(data_x_without,data_y[i],test_size=0.2, random_state=30,shuffle=True)
    ad_without = AdaBoostRegressor()
    x_train_wt, y_train_wt = ADASYN(sampling_strategy='minority').fit_resample(x_train_wt,y_train_wt)
    ad_without.fit(x_train_wt, y_train_wt)
    ad_y_without_predict = ad_without.predict(x_test_wt)
    error_sum_without_ad+=MAPE(y_test_wt,ad_y_without_predict)
    error_without_ad.append(MAPE(y_test_wt,ad_y_without_predict))

In [32]:
print(error_sum_ad/9)
print(error_sum_without_ad/9)

0.4213626206537173
0.42713446967562063


In [41]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',  # 设置提升类型
    'objective': 'mape',  # 目标函数
    'metric': {'mape'},  # 评估函数
    'num_leaves': 50,  # 叶子节点数
    'learning_rate': 0.02,  # 学习速率
    'feature_fraction': 1,  # 建树的特征选择比例
    'bagging_fraction': 1,  # 建树的样本采样比例
    'bagging_freq': 5,  # k 意味着每 k 次迭代执行bagging
    'verbose': 1  # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
}
error_sum_lgb=0
error_lgb=[]
error_sum_without_lgb=0
error_without_lgb=[]
for i in target_list:
    x_train, x_test ,y_train,y_test=train_test_split(data_x,data_y[i],test_size=0.2, random_state=30,shuffle=True)
    x_train, y_train = ADASYN(sampling_strategy='minority').fit_resample(x_train,y_train)
    lgb_train = lgb.Dataset(x_train, y_train)
    gbm = lgb.train(params, lgb_train, num_boost_round=100)
    lgb_y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)
    error_sum_lgb+=MAPE(y_test,lgb_y_pred)
    error_lgb.append(MAPE(y_test,lgb_y_pred))
    x_train_wt, x_test_wt ,y_train_wt,y_test_wt=train_test_split(data_x_without,data_y[i],test_size=0.2, random_state=30,shuffle=True)
    x_train_wt, y_train_wt = ADASYN(sampling_strategy='minority').fit_resample(x_train_wt,y_train_wt)
    lgb_train_wt = lgb.Dataset(x_train_wt, y_train_wt)
    gbm_wt = lgb.train(params, lgb_train_wt, num_boost_round=100)
    lgb_y_pred_wt = gbm_wt.predict(x_test_wt, num_iteration=gbm_wt.best_iteration)
    error_sum_without_lgb+=MAPE(y_test_wt,lgb_y_pred_wt)
    error_without_lgb.append(MAPE(y_test_wt,lgb_y_pred_wt))

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000419 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1154
[LightGBM] [Info] Number of data points in the train set: 618, number of used features: 38
[LightGBM] [Info] Start training from score 3.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000180 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 320
[LightGBM] [Info] Number of data points in the train set: 603, number of used features: 35
[LightGBM] [Info] Start training from score 2.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000385 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 508
[LightGBM] [Info] Number of data points in the train set: 516, number of used features: 39
[LightGBM] [Info] Start training from score 2.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [42]:
print(error_sum_lgb/9)
print(error_sum_without_lgb/9)

0.40220671154922727
0.4061362950753337


In [43]:
print(error_rfr)
print(error_without_rfr)
print(error_ad)
print(error_without_ad)
print(error_lgb)
print(error_without_lgb)

[0.37489408099688476, 0.43155763239875383, 0.3560919003115264, 0.389898753894081, 0.3937601246105919, 0.3782242990654205, 0.6399844236760126, 0.40953115264797507, 0.4215685358255452]
[0.361993769470405, 0.3864953271028037, 0.3602841381100727, 0.4017803738317757, 0.4117145638629283, 0.3977703011422637, 0.6333895379023884, 0.4302345534787124, 0.4244997403946002]
[0.39250151776553244, 0.3927563818194927, 0.3442586942671368, 0.3772428613348637, 0.38523088001359307, 0.37704012015483285, 0.6957492612505136, 0.40757170184581887, 0.4199121674316721]
[0.39019300510333527, 0.4098040150732315, 0.346403576366931, 0.397367820569699, 0.3876256113713539, 0.4040083492268512, 0.6565444911378554, 0.43033110347523534, 0.4219322547560933]
[0.35363052148038043, 0.4179956613009347, 0.3726515987101886, 0.3813513197281611, 0.36493143732279315, 0.38375723611781876, 0.5437601116170501, 0.409331799912711, 0.3924507177530073]
[0.35361536978232894, 0.40036664618713635, 0.3774012975997556, 0.36195997263541707, 0.37

In [44]:
res_rfr=[error_rfr,error_without_rfr]
res_rfr=pd.DataFrame(np.array(res_rfr).T)
res_ad=[error_ad,error_without_ad]
res_ad=pd.DataFrame(np.array(res_ad).T)
res_lgb=[error_lgb,error_without_lgb]
res_lgb=pd.DataFrame(np.array(res_lgb).T)

In [45]:
res_rfr.columns=['with','without']
res_ad.columns=['with','without']
res_lgb.columns=['with','without']
res_rfr.index=target_list
res_ad.index=target_list
res_lgb.index=target_list

In [46]:
res_rfr.to_csv(r'.\data\res_rfr_ADASYN.csv',index=True)
res_ad.to_csv(r'.\data\res_ad_ADASYN.csv',index=True)
res_lgb.to_csv(r'.\data\res_lgb_ADASYN.csv',index=True)